In [ ]:
# start Milvus server
!curl https://raw.githubusercontent.com/milvus-io/milvus/master/deployments/docker/standalone/docker-compose.yml > docker-compose.yml
!docker-compose down
!docker-compose up -d

In [ ]:
# install dependencies
!python3 -m pip install -r requirements.jupyter.txt

In [ ]:
from notebook_config import UPLOAD_PATH, SEARCH_FEATURE_PATH, LOAD_FEATURE_PATH, METRIC_TYPE, MAX_FACES, NUM_KERNEL,SIGMA,AGGREGATION_METHOD,WEIGHTS,CUDA_DEVICE

In [ ]:
import numpy as np

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.utils.data as data

from milvus3d.MeshNet import MeshNet
from milvus3d.transform import Transformer
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

import subprocess
import os
import csv

from collections import deque
import functools
import warnings
from IPython.display import Image
# warnings.filterwarnings('ignore')

In [ ]:
from numpy import linalg as LA

class Encode:
    """
    Create embedding vector for a 3d model

    Input <str>: path of the preprocessed 3d model in npy format
    Output <List>: normalized embedding vector for that 3d model
    """

    def do_extract(self, path, transformer):
        data = self.prepare(path)
        return self.extract_fea(transformer, *data)

    def extract_fea(self, transformer, centers, corners, normals, neighbor_index):
        if torch.cuda.is_available():
            centers = Variable(torch.cuda.FloatTensor(centers.cuda()))
            corners = Variable(torch.cuda.FloatTensor(corners.cuda()))
            normals = Variable(torch.cuda.FloatTensor(normals.cuda()))
            neighbor_index = Variable(torch.cuda.LongTensor(neighbor_index.cuda()))
        else:
            centers = Variable(torch.FloatTensor(centers.cpu()))
            corners = Variable(torch.FloatTensor(corners.cpu()))
            normals = Variable(torch.FloatTensor(normals.cpu()))
            neighbor_index = Variable(torch.LongTensor(neighbor_index.cpu()))
        # get vectors
        feat = list(transformer.get_vector(centers, corners, normals, neighbor_index).tolist())
        return feat / LA.norm(feat)

    def prepare(self, path):
        data = np.load(path)
        face = data['faces']
        neighbor_index = data['neighbors']

        # fill for n < max_faces with randomly picked faces
        num_point = len(face)
        if num_point < 1024:
            fill_face = []
            fill_neighbor_index = []
            for i in range(MAX_FACES - num_point):
                index = np.random.randint(0, num_point)
                fill_face.append(face[index])
                fill_neighbor_index.append(neighbor_index[index])
            face = np.concatenate((face, np.array(fill_face)))
            neighbor_index = np.concatenate((neighbor_index, np.array(fill_neighbor_index)))

        # to tensor
        face = torch.from_numpy(face).float()
        neighbor_index = torch.from_numpy(neighbor_index).long()

        # reorganize
        face = face.permute(1, 0).contiguous()
        centers, corners, normals = face[:3], face[3:12], face[12:]
        corners = corners - torch.cat([centers, centers, centers], 0)

        return centers[np.newaxis, :, :], corners[np.newaxis, :, :], normals[np.newaxis, :, :], neighbor_index[np.newaxis, :, :]


In [ ]:
def get_models(path):
    models = []
    for f in os.listdir(path):
        if ((f.endswith(extension) for extension in
             ['npy']) and not f.startswith('.DS_Store')):
            models.append(os.path.join(path, f))
    return models

def extract_features(model_dir, transformer):
    feats = []
    names = []
    model_list = get_models(model_dir)


    total = len(model_list)
    model = Encode()
    for i, model_path in enumerate(model_list):
        if i%1001 == 1000:
            print(f"Extracting features: {i} out of {total}")
        # create embedding for model
        norm_feat = model.do_extract(model_path, transformer)
        feats.append(norm_feat.tolist())
        names.append(model_path.encode())

    return feats, names


## Download Dataset

In [ ]:
!pip install gdown
import gdown


target_dir = UPLOAD_PATH + ".tar.gz"
if UPLOAD_PATH == "test_data":
    data_gdown_path = "https://drive.google.com/uc?id=1m0fRU6RZG1zi2cZIDpAp8a1uOpAs9Wi-"
elif UPLOAD_PATH == "ModelNet40":
    data_gdown_path = "https://drive.google.com/uc?id=1iJNcFliFL7zEmroBHR0iH0a40lVQ8pDR"
    
if not os.path.exists('data/' + UPLOAD_PATH):
    !gdown "{data_gdown_path}" -O {target_dir}
    !tar -xf {target_dir} -C data/
    !rm {target_dir}
    
# preprocess


In [ ]:
# run this if you wish to preprocess it locally
# !cd data && ./preprocess.sh true

In [ ]:
if UPLOAD_PATH == "ModelNet40":
    if not os.path.exists('data/load_feature'):

        !gdown "https://drive.google.com/uc?id=1XFonx5ubCSTzEQGvGkpX5LXgdAK3yHQX" -O data/load_feature.tar.gz
        !tar -xf data/load_feature.tar.gz -C data
        !rm data/load_feature.tar.gz
        
elif UPLOAD_PATH == 'test_data':
    !cd data && ./preprocess.sh true

## Load DL Model

In [ ]:

weights_dir = "models/"
weights = "MeshNet_best_9192.pkl"
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)
if not os.path.exists(weights_dir + weights):
    !gdown "https://drive.google.com/uc?id=1t5jyJ4Ktmlck6GYhNTPVTFZuRP7wPUYq" -O {weights_dir+weights}


In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = CUDA_DEVICE

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = MeshNet(require_fea=False)
model = nn.DataParallel(model)

model.load_state_dict(torch.load(WEIGHTS, map_location=device))
model.to(device)

model.eval()

## Create Embedding

In [ ]:
transformer = Transformer(model)


In [ ]:
if not os.path.exists("data/vectors.txt"):
    vectors, names = extract_features('data/' + LOAD_FEATURE_PATH, transformer)
    np.savetxt("data/vectors.txt", np.array(vectors), delimiter=',')
    np.savetxt("data/names.txt",np.array(names), delimiter=',', fmt="%s")
else:
    
    with open("data/names.txt", newline='') as f:
        reader = csv.reader(f)
        names = list(reader)
        names = [i[0] for i in names]
    
    vectors = np.genfromtxt("data/vectors.txt",delimiter=',').tolist()


## Connect to Milvus Server

In [ ]:
connections.connect(host="localhost", port=19530)

## Create Collection

In [ ]:
# Delete the collection if it exists
collection_name = "mesh_similarity_search"
if utility.has_collection(collection_name):
    collection = Collection(name=collection_name)
    collection.drop()

In [ ]:
dim = 256
default_fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=dim)
]
default_schema = CollectionSchema(fields=default_fields, description="3d model test collection")

collection = Collection(name=collection_name, schema=default_schema)

## Insert

In [ ]:
import time 
t = time.time()

mr = collection.insert([[i for i in vectors]])

t = time.time() - t
ids = mr.primary_keys

print(f'Inserting {len(ids)} vectors took {t} seconds.')

## Create Index

In [ ]:
default_index = {"index_type": "IVF_SQ8", "metric_type": METRIC_TYPE, "params": {"nlist": 16384}}
t = time.time()
status = collection.create_index(field_name="vector", index_params=default_index)
t = time.time() - t
print(f'Creating index for {len(ids)} vectors took {t} seconds.')

## Load

In [ ]:
collection.load()

## Store mapping
Here we need to store mapping that maps Milvus_id returned by Milvus vector database to Embedding vectors

In [ ]:
# store {milvus_id: embedded vector} in a python dictionary
milvus_to_vector = {}
for i in range(len(names)):
    milvus_to_vector[ids[i]] = vectors[i]

In [ ]:
# record milvus id
def store_milvus_id(ids, root):
    result = {}
    d_list = deque(ids)
    data_root = 'data/' + LOAD_FEATURE_PATH
    for filename in os.listdir(data_root):
        if ((f.endswith(extension) for extension in
             ['npy']) and not filename.startswith('.DS_Store')):
            result[d_list[0]] = root+'/' +filename.split('.')[0]+ ".off"
            d_list.popleft()

    assert not d_list
    return result

In [ ]:
milvus_to_filename = store_milvus_id(ids, 'data/' + UPLOAD_PATH)

## Search

In [ ]:
# Select a 3d model
search_model_path = UPLOAD_PATH + "/toilet_0001.off"
search_filename = search_model_path.split('/')[-1]
search_path = '/'.join(search_model_path.split('/')[:-1])

In [ ]:
# Preprocess
!cd data && python3 compress.py --batch "F" --filename {search_filename} --path {search_path}
!docker run -it --rm -v `pwd`:/data pymesh/pymesh /bin/bash -c "cd /data/data && python preprocess_npy.py --batch 'F' --filename {search_filename}"

    

In [ ]:
encoder = Encode()
feat = encoder.do_extract(os.path.join('data/'+SEARCH_FEATURE_PATH, search_filename.replace("off","npz")), transformer)

In [ ]:

search_params = {"metric_type": METRIC_TYPE, "params": {"nprobe": 16}}
t = time.time()
res = collection.search([feat.tolist()], anns_field="vector", param=search_params, limit=3)
t = time.time() - t
print(f'Searching one vector in a vector database that has {len(ids)} vectors took {t} seconds.')

In [ ]:
# Parse results
vids = [x.id for x in res[0]]
paths = [milvus_to_filename[vids[i]] for i in range(len(vids))]
distances = [x.distance for x in res[0]]

## Display results

In [ ]:
from pygel3d import hmesh, gl_display as gl
from pygel3d import jupyter_display as jd

model = hmesh.load('data/'+search_model_path)
print("filename: " + search_model_path.split('/')[-1])
jd.set_export_mode(True)
jd.display(model, smooth=False)

In [ ]:
# display returned 3d models
for return_path in paths[1:]:
    model = hmesh.load(return_path)
    print("filename: " + return_path.split('/')[-1])
    jd.set_export_mode(True)
    jd.display(model, smooth=False)

## Demo for the model compression process
This solution compresses the models with n faces to 1024 faces to:
1. Save computatioal resources
2. Let the DL model focus more on the structure rather than detailed features.

In [ ]:
# Select a 3d model
search_model_path = UPLOAD_PATH + "/airplane_0001.off"
search_filename = search_model_path.split('/')[-1]
search_path = '/'.join(search_model_path.split('/')[:-1])

# Compression
!cd data && python3 compress.py --batch "F" --filename {search_filename} --path {search_path}


In [ ]:
from pygel3d import hmesh, gl_display as gl
from pygel3d import jupyter_display as jd

model = hmesh.load('data/' + search_model_path)
print("This is the original 3d Model")
print("filename: " + search_model_path.split('/')[-1])
jd.set_export_mode(True)
jd.display(model, smooth=False)

In [ ]:
print("This is the 3d Model after compression.")
model = hmesh.load('data/'+search_model_path.replace(UPLOAD_PATH,'search_feature'))
print("filename: " + search_model_path.split('/')[-1])
jd.set_export_mode(True)
jd.display(model, smooth=False)